In [1]:
# import packages
import pandas as pd
import numpy as np
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys
sys.path.append('../../../open-grid-emissions/src/')

import download_data
import load_data
from column_checks import get_dtypes
from filepaths import *
import impute_hourly_profiles
import data_cleaning
import output_data
import emissions
import validation
import gross_to_net_generation
import eia930
from logging_util import get_logger, configure_root_logger
configure_root_logger()
logger = get_logger("test")


year = 2021
path_prefix = f"{year}/"

In [2]:
crosswalk = load_data.load_pudl_table("epacamd_eia")
crosswalk

,report_year,plant_id_epa,emissions_unit_id_epa,generator_id_epa,plant_id_eia,boiler_id,generator_id
0,2018,3,1,1,3,1,1
1,2018,3,2,2,3,2,2
2,2018,3,3,3,3,3,3
3,2018,3,4,4,3,4,4
4,2018,3,5,5,3,5,5
...,...,...,...,...,...,...,...
12827,2021,63688,CT-9,CTG-9,63688,None,CTG-9
12828,2021,64854,CT1,GTG01,64854,None,GTG01
12829,2021,64854,CT2,GTG02,64854,None,GTG02
12830,2021,70454,MAG1,MAG1,54538,None,MAG1


In [11]:
crosswalk[crosswalk["plant_id_epa"] == 65961]

,report_year,plant_id_epa,emissions_unit_id_epa,generator_id_epa,plant_id_eia,boiler_id,generator_id


In [10]:
oge_cw = load_data.load_epa_eia_crosswalk(year)
oge_cw

,report_year,plant_id_epa,emissions_unit_id_epa,generator_id_epa,plant_id_eia,boiler_id,generator_id
0,2018.0,3,1,1,3,1,1
1,2021.0,3,1,1,3,1,1
2,2018.0,3,2,2,3,2,2
3,2021.0,3,2,2,3,2,2
4,2018.0,3,3,3,3,3,3
...,...,...,...,...,...,...,...
38837,NaN,65961,NaN,NaN,65961,NaN,GEN1
38838,NaN,65962,NaN,NaN,65962,NaN,MS4
38839,NaN,65962,NaN,NaN,65962,NaN,MS4B
38840,NaN,65963,NaN,NaN,65963,NaN,MS5


In [13]:
pudl_subplants = load_data.load_pudl_table("epacamd_eia_subplant_ids")
pudl_subplants

,plant_id_eia,plant_id_epa,subplant_id,unit_id_pudl,emissions_unit_id_epa,generator_id
0,1,1.0,0,NaN,WT2,WT2
1,1,1.0,1,NaN,WT1,WT1
2,1,1.0,2,NaN,5.1,5.1
3,1,1.0,3,NaN,5,5
4,1,1.0,4,NaN,3,3
...,...,...,...,...,...,...
39943,880109,NaN,0,NaN,B001,None
39944,880110,NaN,0,NaN,1,None
39945,880110,NaN,1,NaN,2,None
39946,880110,NaN,2,NaN,3,None


In [15]:
pudl_subplants[pudl_subplants["plant_id_epa"] == 1732]

,plant_id_eia,plant_id_epa,subplant_id,unit_id_pudl,emissions_unit_id_epa,generator_id
4716,1732,1732.0,0,NaN,10,7
4717,1732,1732.0,0,NaN,10,8
4718,1732,1732.0,0,NaN,11,7
4719,1732,1732.0,0,NaN,12,7
4720,1732,1732.0,0,NaN,9,7
4721,1732,1732.0,0,NaN,11,8
4722,1732,1732.0,0,NaN,12,8
4723,1732,1732.0,0,NaN,9,8
4724,1732,1732.0,1,NaN,6,6


In [16]:
crosswalk[crosswalk["plant_id_epa"] == 1732]

,report_year,plant_id_epa,emissions_unit_id_epa,generator_id_epa,plant_id_eia,boiler_id,generator_id
1264,2018,1732,10,7,1732,10,7
1265,2018,1732,10,8,1732,10,8
1266,2018,1732,11,7,1732,11,7
1267,2018,1732,11,8,1732,11,8
1268,2018,1732,12,7,1732,12,7
1269,2018,1732,12,8,1732,12,8
1270,2018,1732,9,7,1732,9,7
1271,2018,1732,9,8,1732,9,8
7683,2021,1732,10,7,1732,10,7
7684,2021,1732,10,8,1732,10,8
